# PV Degradation Tools Tutorial - 2
### Degradation and Solder Fatigue
***
Matthew Brown

2023.06.02
***

**Requirements**:
- compatible weather file (PSM3, TMY3, EPW)
- Accelerated testing chamber parameters
    - chamber irradiance [W/m^2]
    - chamber temperature [C]
    - chamber humidity [%]
- Activation energies for test material [kJ/mol]

**Objectives**:
1. Read in the weather data
2. Generate basic modeling data
3. Calculate VantHoff degradation acceleration factor
4. Calculate Arrhenius degradation acceleration factor
5. Solder Fatigue

In [1]:
import os
import pandas as pd

import pvdeg 
from pvdeg import TEST_DATA_DIR

## 1. Read In the Weather File

This is usually the first step. Use a PSM3, TMY3, or EPW file. For this demo, use the provided PSM3 weather file.

In [2]:
PSM_FILE = os.path.join(TEST_DATA_DIR,'psm3_pytest.csv')
WEATHER, META = pvdeg.weather.read(PSM_FILE,'psm')

## 2. Generate Basic Modeling Data

For this tutorial we will need solar position, POA, PV cell and module temperature.

In [3]:
sol_pos = pvdeg.spectral.solar_position(weather_df=WEATHER, meta=META)

poa_df = pvdeg.spectral.poa_irradiance(weather_df=WEATHER, meta=META, solar_position=sol_pos)

temp_cell = pvdeg.temperature.cell(weather_df=WEATHER, poa=poa_df)

temp_module = pvdeg.temperature.module(weather_df=WEATHER, poa=poa_df)

## 3. VantHoff Degradation

Van 't Hoff Irradiance Degradation

For one year of degredation the controlled environmnet lamp settings will need to be set to IWa.

In [5]:
# chamber irradiance (W/m^2)
I_chamber = 1000 
# chamber temperature (C)
temp_chamber = 60

# calculate the VantHoff Acceleration factor
vantHoff_deg = pvdeg.degradation.vantHoff_deg(I_chamber=I_chamber, poa=poa_df,
                                              temp_cell=temp_cell, temp_chamber=temp_chamber)

# calculate the VantHoff weighted irradiance
irr_weighted_avg_v = pvdeg.degradation.IwaVantHoff(poa=poa_df, temp_cell=temp_cell)

## 4. Arrhenius
Calculate the Acceleration Factor between the rate of degredation of a modeled environmnet versus a modeled controlled environmnet

Example: "If the AF=25 then 1 year of Controlled Environment exposure is equal to 25 years in the field"

Function to calculate IWa, the Environment Characterization (W/m²). For one year of degredation the controlled environmnet lamp settings will need to be set at IWa.

In [7]:
# relative humidity within chamber (%)
rh_chamber = 15
# arrhenius activation energy (kj/mol)
Ea = 40

rh_surface = pvdeg.humidity.rh_surface_outside(rh_ambient=WEATHER['relative_humidity'],
                                               temp_ambient=WEATHER['temp_air'],
                                               temp_module=temp_module)

arrhenius_deg = pvdeg.degradation.arrhenius_deg(I_chamber=I_chamber, rh_chamber=rh_chamber,
                                                rh_outdoor=rh_surface, poa=poa_df,
                                                temp_chamber=temp_chamber, temp_cell=temp_cell,
                                                Ea=Ea)

irr_weighted_avg_a = pvdeg.degradation.IwaArrhenius(poa=poa_df, 
                                                    rh_outdoor=WEATHER['relative_humidity'],
                                                    temp_cell=temp_cell, Ea=Ea)